# Defendant's Fallacy

For more, see this very thorough [treatment](https://www.untrammeledmind.com/2018/12/defense-attorneys-fallacy-a-conditional-probability-problem/) of this problem.

In [1]:
%load_ext autoreload
%autoreload 2

*Note to the Reader*: The above code leverages a "magic" function native to IPython that allows the changes to imported modules to be immediately reflected in the notebook environment. It need only be run once per session.

In [2]:
import numpy as np
import pandas as pd
from custom.defendants_fallacy import data_generator, data_tests

In [3]:
# Generate and load data
df = data_generator()
data_tests(df)

No errors!


Note, we fabricated data on a number of couples, $N = 5,000,000$, and assume that the unconditional probability of a wife being murdered to be $P(M) = 0.0005$.

## Motivation 👨🏼‍⚖️

Consider this quote from *Calculated Risks: How to Know When Numbers Deceive You* (Gigerenzer, 2002) regarding the proceedings of the infamous trial of OJ Simpson: 
    
> …the prosecution presented evidence that Simpson had been violent toward his wife, while the defense argued that there was only one woman murdered for every 2,500 women who were subjected to spousal abuse, and that any history of Simpson being violent toward his wife was irrelevant to the trial.

### Question

*Should* the fact that OJ had been violent toward his wife be thrown out on the grounds of irrelevance (i.e., what the defense asserts)? 

To answer this question, let's use data. Here's what we know using the data on hand and what's been said in court:

- A woman has been murdered, and her husband is accused of having committed the murder. 
- It is known that the man abused his wife repeatedly in the past, and the prosecution argues that this is important evidence pointing towards the man’s guilt. The defense attorney says that the history of abuse is irrelevant, as only 1-in-2500 men who beat their wives end up murdering them.
- Assume that the defense attorney is not committing perjury and the 1-in-2500 figure is correct.
- Our data tell us that half of men who murder their wives previously abused them. 
- Our data also tell us that 20% of murdered married women were killed by their husbands, and that if a woman is murdered and the husband is not guilty, then there is only a 10% chance that the husband abused her. 

## Analysis

Below, we start down the path of trying to figure out whether it matters that the husband abused his wife or not. 

### Step 0

We need to define some events. (Hint: Look at the column headers in our data.) Begin to think about Bayes' Rule...🤔

In [4]:
df.columns

Index(['M', 'G', 'A'], dtype='object')

In [5]:
for col in df:
    display(df[col].unique())

array([0, 1])

array([0, 1])

array([1, 0])

These columns all look like *indicator* random variables. We can also look at the relative frequencies, or the *probabilities*, of each of these events:

In [6]:
df.mean()

M    0.0005
G    0.0001
A    0.1250
dtype: float64

In [7]:
## TODO: Explore the data on your more if you desire.

### Step 1 

Inspect the data. What does it mean? Can you confirm the facts drawn from our data (see above)?

- Show that the defense attorney is correct, i.e., $P(G|A) = \frac{1}{2500}$.
- Show that half of men who murder their wives previously abused them. 
- Show that 20% of murdered married women were killed by their husbands
- Show that if a woman is murdered and the husband is not guilty, then there is only a 10% chance that the husband abused her. 

In [8]:
## TODO: Confirm the facts of the case using the data.

p_G_given_A = df.query("A == 1")['G'].mean()

p_A_given_G_and_M = df.query("G == 1 and M == 1")['A'].mean()

p_G_given_M = df.query("M == 1")['G'].mean()

p_A_given_notG_and_M = df.query("G == 0 and M == 1")['A'].mean()

In [9]:
print(f"P(G|A) = {p_G_given_A:.5f}")
print(f"P(A|G,M) = {p_A_given_G_and_M:.5f}")
print(f"P(G|M) = {p_G_given_M:.5f}")
print(f"P(A|¬G,M) = {p_A_given_notG_and_M:.5f}")

P(G|A) = 0.00040
P(A|G,M) = 0.50000
P(G|M) = 0.20000
P(A|¬G,M) = 0.10000


### Step 2 

Hope you've been pondering Bayes' Theorem. What is the _prior_ probability that the husband is guilty of murdering his wife?

- Calculate $P(G|M)$.

In [10]:
## TODO: Calculate the prior probability of guilt given the wife has been murdered.
##       Ask yourself if you've already done this calculation...

We will need to compare our *posterior* to this to see if the husband is more likely of guilt. If so, that implies that the defense attorney is trying to pull the wool over the jury's eyes!

### Step 3

Figure out what the posterior probability of guilt is.

- You know that the posterior probability is $P(G|...)$. 
- Think about what event(s) to condition on (i.e., what evidence do we have?).
- Drawing a tree diagram might help.

In [11]:
## TODO: Calculate the posterior probability of guilt. 
##       Do this either analytically or numerically (directly on the data)

## Solution

The grave error the defense is making is that they are not considering *all* of the available evidence: namely, that the wife was murdered!

### Analytical

Let $A$, $G$, and $M$ be the events that that wife was abused, the husband is guilty, and that the wife was murdered, respectively. 

Given:

$P(G|A) = \frac{1}{2500}$, provided by the defense<br>
$P(G|M) = 0.2$, provided by our data<br>
$P(A|G,M) = 0.5$, provided by our data<br>
$P(A|G^{c},M) = 0.1$, provided by our data<br>

We can directly use Bayes' Thereom with "extra" conditioning:

\begin{align}
P(G|A,M) & = \frac{P(A|G,M)P(G|M)}{P(A|G,M)P(G|M) + P(A|G^{c},M)P(G^{c}|M)} \\
P(G|A,M) & = \frac{(0.5 \cdot 0.2)}{(0.5 \cdot 0.2) + (0.1 \cdot 0.8)} \\
P(G|A,M) & = \frac{5}{9}
\end{align}

In [12]:
# Events
G_and_M = "G == 1 and M == 1"
notG_and_M = "G == 0 and M == 1"
M = "M == 1"
G = "G == 1"

# Terms in Bayes Rule
p_A_given_G_and_M = df.query(G_and_M)['A'].mean()
p_G_given_M = df.query(M)['G'].mean()
p_A_given_notG_and_M = df.query(notG_and_M)['A'].mean()
p_notG_given_M = 1 - p_G_given_M

# Answer 
p_G_given_A_and_M =\
    (p_A_given_G_and_M * p_G_given_M) /\
    ((p_A_given_G_and_M * p_G_given_M) + (p_A_given_notG_and_M * p_notG_given_M))

p_G_given_A_and_M

0.5555555555555555

Notice that when we incorporate *all* of the evidence available to us, our *posterior* belief that the husband is guilty rises to over 50% (versus our *prior* belief of 20%).

Notice that we also *never used* the defense attorney's information in our calculation -- we don't care about the proportion of abusive husbands who murder their wives (i.e., $P(G|A)$)...instead, we care about the proportion of abusive husbands who murder their wives taken from the group of abusive husbands whose wives were murdered (i.e., $P(G|A,M)$)!

**While most cases of spousal abuse do not end in murder, most cases of murder when there is a history of spousal abuse can be attributed to the spouse.**

Of course, since we had a comprehensive view of the sample space via the data on hand, we also could've conditioned on $A$ first, then applied Bayes Theorem under that condition:

\begin{align}
P(G|A,M) & = \frac{P(M|G,A)P(G|A)}{P(M|G,A)P(G|A) + P(M|G^{c},A)P(G^{c}|A)} \\
P(G|A,M) & = \frac{(1 \cdot 0.0004)}{(1 \cdot 0.0004) + (0.000320128 \cdot 0.9996)} \\
P(G|A,M) & = \frac{5}{9}
\end{align}

However, the facts of the case, as presented, didn't supply $P(M|G^c,A)$. In fact, if you look at how Ravi implemented the `data_generator()` function, you can decide on any "small" $P(G|A)$ and $P(M)$ and get the same result: $P(M|G^c, A)$ will adjust to changes in choice of $P(G|A)$ and $P(M)$ and the conditions stipulated above, yielding the same ratio $\frac{5}{9}$.

In [13]:
# Events
G_and_A = "G == 1 and A == 1"
notG_and_A = "G == 0 and A == 1"
A = "A == 1"
G = "G == 1"

# Terms in Bayes Rule
p_M_given_G_and_A = df.query(G_and_A)['M'].mean()
p_G_given_A = df.query(A)['G'].mean()
p_M_given_notG_and_A = df.query(notG_and_A)['M'].mean()
p_notG_given_A = 1 - p_G_given_A

# Answer 
p_G_given_A_and_M =\
    (p_M_given_G_and_A * p_G_given_A) /\
    ((p_M_given_G_and_A * p_G_given_A) + (p_M_given_notG_and_A * p_notG_given_A))

p_G_given_A_and_M

0.5555555555555556

The conclusion is the same: the evidence that OJ abused his wife is not irrelevent...it's _very_ relevant and should not be thrown out (despite the protestations of the defense)! What's irrelevant is the number cited by the defense that makes it seem that being an abusive husband is not important.

### Numerical

We can get the same solution directly from the data:

In [14]:
A_and_M = "A == 1 and M == 1"

# Answer
df.query(A_and_M)['G'].mean()

0.5555555555555556